### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

In [12]:
# Fechas
today = datetime.date.today()
ilm = str((today - relativedelta(months=1)).replace(day=1))
flm = str(today - relativedelta(days=today.day))
# Ciudades
city_growth = "'Río Gallegos','Comodoro Rivadavia','Ushuaia','Puerto Madryn','Trelew','Carmen de Patagones','Viedma'"

### QUERIES

In [13]:
# 714MB
q_orders = '''SELECT c.city_name AS City,
       COUNT(DISTINCT o.order_id) AS Total,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.order_id ELSE NULL END) AS Logistic,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('REJECTED','PENDING','CANCELLED') AND o.fail_rate_owner = 'Restaurant' THEN o.order_id ELSE NULL END) AS Rejected_Vendor
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.country_id = 3
      AND o.address.area.city.name IN ({2})
GROUP BY 1'''.format(ilm,flm,city_growth)

# 6MB
q_online = '''SELECT p.city.name AS City,
       COUNT(DISTINCT CASE WHEN pm.is_online THEN pm.restaurant_id ELSE NULL END) AS Online,
       COUNT(DISTINCT CASE WHEN pm.is_online AND pm.has_confirmed_orders = FALSE THEN pm.restaurant_id ELSE NULL END) AS Zombies,
       COUNT(DISTINCT CASE WHEN pm.is_churned THEN pm.restaurant_id ELSE NULL END) AS Churns
FROM `peya-bi-tools-pro.il_core.fact_partners_monthly` AS pm
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON pm.restaurant_id = p.partner_id
WHERE DATE(pm.full_date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND p.country_id = 3
      AND p.city.name IN ({2})
GROUP BY 1'''.format(ilm,flm,city_growth)

# 970MB
q_campaigns = '''SELECT c.city_name AS City,
       COUNT(DISTINCT o.order_id) AS Confirmed_Mktg
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
UNNEST (details) AS od
LEFT JOIN `peya-bi-tools-pro.il_core.dim_subsidized_product` AS sp ON od.product.product_id = sp.product_id AND od.is_subsidized
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_subsidized_campaign` AS sc ON sp.subsidized_product_campaing_id = sc.subsidized_campaign_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND od.is_subsidized
      AND o.order_status = 'CONFIRMED'
      AND o.country_id = 3
      AND sc.subsidized_campaign_id IS NOT NULL
      AND o.address.area.city.name IN ({2})
GROUP BY 1'''.format(ilm,flm,city_growth)

In [14]:
# Descargo la data
hue_orders = pd.io.gbq.read_gbq(q_orders, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.37rows/s]


In [15]:
hue_online = pd.io.gbq.read_gbq(q_online, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  6.14rows/s]


In [16]:
hue_campaigns = pd.io.gbq.read_gbq(q_campaigns, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.49rows/s]


In [17]:
# Copio las bases
orders = hue_orders.copy()
online = hue_online.copy()
campaigns = hue_campaigns.copy()

### TRABAJO

In [18]:
# Hago un Merge de las tablas
cm_growth = orders.merge(campaigns,on=['City'],how='outer')
cm_growth = cm_growth.merge(online,on=['City'],how='outer')
cm_growth.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [19]:
# Creo las columnas faltantes
cm_growth['%Campaigns'] = cm_growth['Confirmed_Mktg'] / cm_growth['Confirmed']
cm_growth['%OD'] = cm_growth['Logistic'] / cm_growth['Confirmed']
cm_growth['VFR'] = cm_growth['Rejected_Vendor'] / cm_growth['Total']
cm_growth['%Churns'] = cm_growth['Churns'] / cm_growth['Online']
cm_growth['%Cartera Activa'] = (cm_growth['Online'] - cm_growth['Zombies']) / cm_growth['Online']
cm_growth.replace([np.nan,np.inf,-np.inf],0,inplace=True)

### CARGA

In [22]:
# Carga Crudo CM
sheet_id = '1b82zsRm1m3dgbaaVlCRm0n-y7tEK79yXJTLAr8Jfqpo'
wks_name = 'Crudo CM'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(cm_growth, index=False, sheet=wks_name, replace=True)